In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\ML-Workflow-with-DVC-Pipelines\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\ML-Workflow-with-DVC-Pipelines'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir :Path
    source_URL : str
    Local_data_file :Path
    unzip_dir :Path

In [16]:
from chicken_disease_detection.constants import *
from src.chicken_disease_detection.utils.common import read_yaml, create_directories
